In [ ]:
%%capture
!pip install jupyter-dash dash-bootstrap-components
# !pip install dash_loading_spinners
!pip install -q kaggle
!gdown https://drive.google.com/u/0/uc?id=1LbqfXXwufpStoioR1zELC9SBPbbt5JHC

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sudalairajkumar/cryptocurrencypricehistory
!unzip /content/cryptocurrencypricehistory.zip -d dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1d_tqi-Rj4-dOLdv60pEdjpSeujbV817V

Downloading...
From: https://drive.google.com/uc?id=1d_tqi-Rj4-dOLdv60pEdjpSeujbV817V
To: /content/crypto_logo.png
100% 34.9k/34.9k [00:00<00:00, 48.1MB/s]


In [ ]:
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd 
from dash import Input, Output, State, dcc, html, ctx
import dash_bootstrap_components as dbc
import plotly.graph_objects as go


text_color ="#a5c0af"
green_background ="rgba(143, 200, 168, 0.1)"

## Coins Images
bitcoin_src = 'https://static.vecteezy.com/system/resources/previews/008/505/801/original/bitcoin-logo-color-illustration-png.png'
cardano_src = 'https://seeklogo.com/images/C/cardano-ada-logo-4B6BADDB43-seeklogo.com.png'
dogecoin_src = 'https://altcoinsbox.com/wp-content/uploads/2023/01/dogecoin-logo.png'
ethereum_src = 'https://assets.stickpng.com/images/5a7593fc64538c292dec1bbf.png'
litecoin_src = 'https://www.xdefi.io/wp-content/uploads/2022/05/logozzzz.png'
tron_src = 'https://cdn.freebiesupply.com/logos/large/2x/tron-logo-png-transparent.png'

#Read Data

In [ ]:
def read_data(coin, path="/content/dataset/"):
    df = pd.read_csv(path+ f"coin_{coin}.csv", parse_dates=["Date"])
    df['Year'] = df['Date'].dt.year
    return df

df = read_data('Bitcoin')
df.head()

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,Year
0,1,Bitcoin,BTC,2013-04-29 23:59:59,147.488007,134.000000,134.444000,144.539993,0.0,1.603769e+09,2013
1,2,Bitcoin,BTC,2013-04-30 23:59:59,146.929993,134.050003,144.000000,139.000000,0.0,1.542813e+09,2013
2,3,Bitcoin,BTC,2013-05-01 23:59:59,139.889999,107.720001,139.000000,116.989998,0.0,1.298955e+09,2013
3,4,Bitcoin,BTC,2013-05-02 23:59:59,125.599998,92.281898,116.379997,105.209999,0.0,1.168517e+09,2013
4,5,Bitcoin,BTC,2013-05-03 23:59:59,108.127998,79.099998,106.250000,97.750000,0.0,1.085995e+09,2013


In [ ]:
df.isnull().sum()

SNo          0
Name         0
Symbol       0
Date         0
High         0
Low          0
Open         0
Close        0
Volume       0
Marketcap    0
Year         0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2991 entries, 0 to 2990
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   SNo        2991 non-null   int64         
 1   Name       2991 non-null   object        
 2   Symbol     2991 non-null   object        
 3   Date       2991 non-null   datetime64[ns]
 4   High       2991 non-null   float64       
 5   Low        2991 non-null   float64       
 6   Open       2991 non-null   float64       
 7   Close      2991 non-null   float64       
 8   Volume     2991 non-null   float64       
 9   Marketcap  2991 non-null   float64       
 10  Year       2991 non-null   int64         
dtypes: datetime64[ns](1), float64(6), int64(2), object(2)
memory usage: 257.2+ KB


#DataFrame for pie chart

In [ ]:
coins=["Bitcoin","Tron","Dogecoin","Cardano","Ethereum","Litecoin"]
df_all = pd.concat([read_data(coin) for coin in coins ])
pie_df=df_all.drop_duplicates(subset=["Name",'Year'],
                     keep='last')
pie_df.sample(5)

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,Year
246,247,Litecoin,LTC,2013-12-31 23:59:59,24.603800,23.709999,24.505100,24.347000,7.446870e+06,5.952194e+08,2013
2437,2438,Bitcoin,BTC,2019-12-31 23:59:59,7335.290051,7169.777686,7294.438896,7193.598978,2.116795e+10,1.304461e+11,2019
820,821,Cardano,ADA,2019-12-31 23:59:59,0.033898,0.032794,0.033607,0.032845,2.532035e+07,8.515764e+08,2019
2572,2573,Dogecoin,DOGE,2020-12-31 23:59:59,0.004743,0.004553,0.004636,0.004682,8.549834e+07,5.981490e+08,2020
246,247,Bitcoin,BTC,2013-12-31 23:59:59,760.580017,738.169983,760.320007,754.010010,2.089730e+07,9.191325e+09,2013


In [ ]:
def generate_currency_card(currency_name, currency_src):
    return dbc.Button(id=f"{currency_name}_button", children=[currency_name, html.Img(src=currency_src, width=40, height=40)],style={"margin": "5px", 'border': 'solid', 'borderWidth': '0px','width':"100"})

currency_cards = html.Div([
    generate_currency_card("Bitcoin", bitcoin_src),
    generate_currency_card("Cardano", cardano_src),
    generate_currency_card("Dogecoin", dogecoin_src),
    generate_currency_card("Ethereum", ethereum_src),
    generate_currency_card("Litecoin", litecoin_src),
    generate_currency_card("Tron", tron_src)
    ], style={'textAlign': 'center'})

def create_dropdown(id, label, options, value):
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.Div(label),
                    dcc.Dropdown(
                        id=id,
                        options=[{'label': str(o), 'value': str(o)} for o in options],
                        value=value
                    )
                ], style={'textAlign': 'center'}) 
            ])
        ),
    ])

def create_graph_card(title, graph_id):
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H2(title,className="card-title",style={"color":text_color}),
                    dcc.Graph(id=graph_id)
                ], style={'textAlign': 'center'}) 
            ])
        ,style={'background-color': green_background}),
    ])


def figure_bigNo(col,df):
    return go.Figure(go.Indicator(
                mode = "number+delta",
                value = round(float(df.iloc[-1][col]),2),
                number = {'prefix': "$","font":{"size":40} , 'font_color':'rgb(85, 114, 170)'},
                delta = {'position': "top", 'reference': round(float(df.iloc[-2][col]),2)},
                domain = {'x': [0, 1], 'y': [0, 1]},
                ),layout=go.Layout(height=100,width=210, paper_bgcolor= 'rgba(1, 1, 1, 0)', plot_bgcolor= 'rgba(1, 1, 1, 0)')
                )
    
def drawLineFigure(df, y_col, title='Line Plot', x_col='Date'):
    return px.scatter(df, x=x_col, y=y_col,title=title,trendline='ols').update_layout(
                template='plotly_dark',
                plot_bgcolor= 'rgba(1, 1, 1, 0.8)',
                paper_bgcolor= 'rgba(1, 1, 1, 0.8)',
                title =  dict(text =title,
                               font =dict(family='Sherif',
                               size=25,
                               color = text_color)),
                               title_x=0.5
            ).update_traces(mode = 'lines')

def drawLPieChart(df):
    return  go.Figure(data=[go.Pie(labels=df['Name'], values=df['Marketcap'], textinfo='label+percent',
                             insidetextorientation='radial'
                            )]).update_layout(
                template='plotly_dark',
                plot_bgcolor= 'rgba(1, 1, 1, 0.8)',
                paper_bgcolor= 'rgba(1, 1, 1, 0.8)',
                title =  dict(text ="Market Capitaliztiion",
                               font =dict(family='Sherif',
                               size=25,
                               color = text_color)),
                               title_x=0.5,
                               title_y=0.1
            )
    
def Candlestick(df):
    return go.Figure(data=[go.Candlestick(
          x=df['Date'],
          open=df['Open'],
          high=df['High'],
          low=df['Low'],
          close=df['Close'])]).update_layout(
                  template='plotly_dark',
                  plot_bgcolor= 'rgba(1, 1, 1, 0.8)',
                  paper_bgcolor= 'rgba(1, 1, 1, 0.8)',
                  xaxis_rangeslider_visible=False,
                  title =  dict(text ="Candlestick",
                               font =dict(family='Sherif',
                               size=25,
                               color = text_color)),
                               title_x=0.5,
                              yaxis_title="value",
                               xaxis_title="Date",
              )
          
def create_card(title, id):
    return dbc.Card(dbc.CardBody([
            html.Div([
                html.H2(title),
                dcc.Graph(id=id)
            ], style={'textAlign': 'center',"border-radius": "90px","border":"solid"}) 
        ]))
    
def get_table(year,coins=coins):
    if year == 'All' or year is None:
        return dbc.Table.from_dataframe(pd.DataFrame([df.iloc[-1][['Name',"Year",'Marketcap']] for df in [read_data(coin) for coin in coins]]), striped=False, bordered=True, hover=False,size="sm", style={'color':text_color,'border-color':'rgb(61, 71, 89)'})
    else:
        return dbc.Table.from_dataframe(pd.DataFrame([df[df.Year==year].iloc[-1][['Name',"Year",'Marketcap']] for df in [read_data(coin) for coin in coins] if year in df.Year.unique()]), striped=False, bordered=True, hover=False, size="sm",style={'color':text_color,'border-color':'rgb(61, 71, 89)'})


#Row0

In [ ]:
Row0 = dbc.Row([
    dbc.Col([
        dbc.Row([
            dbc.Col(
               html.Img(src='https://gcdnb.pbrd.co/images/0UZAudvNnYYv.png?o=1',width=150,height=100),
                    width=12)
        ])
    ], width=5),
    dbc.Col([
        dbc.Row([
          currency_cards
        ], align='center')
    ], width=7)
], align='center')

## Row1

In [ ]:
Row1 = dbc.Row([
    dbc.Col([
        dbc.Row([
            dbc.Col(
              dbc.Card(dbc.CardBody([
              html.Div([
                        html.H2(id="coin_title",style={"color":text_color}),
                        html.Img(id="coin_img",width=90, height=100),
                    ], style={'textAlign': 'center'}) 
                ]),style={"background-color":green_background})
                , 
                    width=12)
        ])
    ], width=3),
    dbc.Col([
        dbc.Row([
            dbc.Col(create_graph_card('Market Cap', 'MarketcapBAN'),width=3),
            dbc.Col(create_graph_card('Volume', 'VolumeBAN'),width=3),
            dbc.Col(create_graph_card('High', 'HighBAN'),width=3),
            dbc.Col(create_graph_card('Low', 'LowBAN'),width=3)
        ], align='center')
    ], width=9)
], align='center')

#Row buttons

In [ ]:
Row_buttons = dbc.Row([
    dbc.Col([
        dbc.Row([
            
        ])
    ], width=3),
    dbc.Col([
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(
                        id='year-dropdown',
                        options=[{'label': str(o), 'value': str(o)} for o in list(df['Year'].unique())+['All']],
                        placeholder="years"
                    )
              ],width=4),
            dbc.Col([dbc.Button("CandleStick", color="secondary", id='CandleStick-Button',style={"margin": "5px"}),
                    dbc.Button("High&Low", color="secondary", id='High&Low-Button',style={"margin": "5px"}),
                    dbc.Button("Open&Close", color="secondary", id='Open&Close-Button',style={"margin": "5px"})
            ],width=8)
        ], align='center')
    ], width=9)
], align='center')

# Row 2

In [ ]:
Row2 = dbc.Row([
    dbc.Col([
        dcc.Graph(id='PieChart')
    ], width=3),

    dbc.Col([
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='VarPlot')
                
            ], width=12)
        ], align='center')
    ], width=9),
], align='center')

# Row3

In [ ]:
Row3 = dbc.Row([
    dbc.Col([ 
        dbc.Card(dbc.CardBody([
            html.Div([
                html.H2('Currencies Marketcap',style={"color":text_color}),
                html.Div(id='table-graph')
            ], style={'textAlign': 'center'}) 
        ]),style={"background-color":'rgba(1,1,1,0.7)'})
        #html.Div()
    ] ,width=3),

    dbc.Col([
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='MarketcapLinePlot')
            ], width=12),
        ], align='center')               
    ], width=9),
], align='center')


# Body

In [ ]:
def create_layout():
    return html.Div([
                Row0,
                html.Br(),
                Row1,
                html.Br(),
                Row_buttons,
                Row2,
                html.Br(),
                Row3
    ],
        style={"background-image": "url(https://rare-gallery.com/uploads/posts/321901-Bitcoin-Cryptocurrency-Cube-4K-3840x2160.jpg)"})
    


In [ ]:
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.config.prevent_initial_callbacks = 'initial_duplicate'

@app.callback(
    Output('MarketcapBAN', 'figure',allow_duplicate=True),
    Output('VolumeBAN', 'figure',allow_duplicate=True),
    Output('HighBAN', 'figure',allow_duplicate=True),
    Output('LowBAN', 'figure',allow_duplicate=True),
    Output('MarketcapLinePlot','figure',allow_duplicate=True),
    Output('PieChart', 'figure',allow_duplicate=True),
    Output('table-graph', 'children'),
    Input('year-dropdown', 'value')
    
)
def update_market_cap(year_value):
    # handling year_value
    if year_value == 'All' or year_value is None:
        plt_df = df
        plt_pie = pie_df
    else:
        year_value = int(year_value)
        plt_df = df[df.Year == year_value]
        plt_pie = pie_df[pie_df.Year ==year_value]
    
    
    # return figures
    cols = ['Marketcap','Volume','High','Low']
    return [figure_bigNo([col],plt_df) for col in cols] + [drawLineFigure(plt_df, 'Marketcap', title='Market Capitaliztiion', x_col='Date')] +[drawLPieChart(plt_pie)] +[get_table(year_value,coins)]


@app.callback(
    Output('VarPlot', 'figure',allow_duplicate=True),
    Output('CandleStick-Button', 'color'),
    Output('High&Low-Button', 'color'),
    Output('Open&Close-Button', 'color'),
    Input('year-dropdown', 'value'),
    Input('CandleStick-Button', 'n_clicks'),
    Input('High&Low-Button', 'n_clicks'),
    Input('Open&Close-Button', 'n_clicks')
)

def update_varPlot(year, *n):
    if year == 'All' or year is None:
        plt_df = df
    else:
        plt_df = df[df.Year == int(year)]
    h=o=c='secondary'
    if "High&Low-Button" == ctx.triggered_id:
        h='success'
        return drawLineFigure(plt_df, ['High','Low'], title='High & Low', x_col='Date'),c,h,o
    elif "Open&Close-Button" == ctx.triggered_id:
        o='success'
        return drawLineFigure(plt_df, ['Open','Close'], title='Open & Close', x_col='Date'),c,h,o
    else:
        c='success'
        return Candlestick(plt_df),c,h,o

@app.callback(
    Output('VarPlot', 'figure',allow_duplicate=True),
    Output('MarketcapBAN', 'figure',allow_duplicate=True),
    Output('VolumeBAN', 'figure',allow_duplicate=True),
    Output('HighBAN', 'figure',allow_duplicate=True),
    Output('LowBAN', 'figure',allow_duplicate=True),
    Output('MarketcapLinePlot','figure',allow_duplicate=True),
    Output('year-dropdown', 'options'),
    Output('coin_title', 'children'),
    Output('coin_img', 'src'),
    Output('Bitcoin_button', 'color'),
    Output('Cardano_button', 'color'),
    Output('Dogecoin_button', 'color'),
    Output('Ethereum_button', 'color'),
    Output('Litecoin_button', 'color'),
    Output('Tron_button', 'color'),
    Input('Bitcoin_button', 'n_clicks'),
    Input('Cardano_button', 'n_clicks'),
    Input('Dogecoin_button', 'n_clicks'),
    Input('Ethereum_button', 'n_clicks'),
    Input('Litecoin_button', 'n_clicks'),
    Input('Tron_button', 'n_clicks'),
)
def update_app(*n):
    global df
    coins=["Bitcoin","Cardano","Dogecoin","Ethereum","Litecoin","Tron"]
    if ctx.triggered_id:
        coin = ctx.triggered_id.split('_')[0]
    else:
        coin = 'Bitcoin'
    figs = list(update_varPlot('All', 5))
    df = read_data(coin, path="/content/dataset/")
    plt_df = df
    colors=["secondary" if c !=coin else "success" for c in coins ]
    cols = ['Marketcap','Volume','High','Low']
    return [Candlestick(plt_df)] + [figure_bigNo([col],plt_df) for col in cols] + [drawLineFigure(plt_df, 'Marketcap', title='Market Capitaliztiion', x_col='Date')] + [[{'label': str(year), 'value': str(year)} for year in list(df['Year'].unique())+['All']]] + [coin,eval(f"{coin.lower()}_src")] +colors
    


app.layout = create_layout()
app.run_server(mode='external')

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>